# 2. Decorators

## Examples

In [1]:
import pytest

@pytest.mark.skip
def test_not_today():
    ...

In [2]:
from dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str

User(id=1, name="Jan")

User(id=1, name='Jan')

In [3]:
from functools import cache
import time

@cache
def very_expensive_function() -> str:
    time.sleep(5)
    return "E == m*c*c"

print(very_expensive_function())
print(very_expensive_function())

E == m*c*c
E == m*c*c


## Lets make one!

### Simple

In [4]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before calling the function.")
        result = func(*args, **kwargs)
        print("After calling the function.")
        return result
    return wrapper

In [5]:
@my_decorator
def say_hello(name):
    return f"Hello, {name}!"

In [6]:
say_hello("World")

Before calling the function.
After calling the function.


'Hello, World!'

In [7]:
say_hello.__name__

'wrapper'

### @wraps

In [8]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Before calling the function.")
        result = func(*args, **kwargs)
        print("After calling the function.")
        return result
    return wrapper

@my_decorator
def say_hello(name):
    return f"Hello, {name}!"

In [9]:
say_hello("World")

Before calling the function.
After calling the function.


'Hello, World!'

In [10]:
say_hello.__name__

'say_hello'

### Experiments

In [11]:
def nope(func):
    def wrapper(*args, **kwargs):
        return None
    return wrapper

In [12]:
@nope
def say_hello(name):
    return f"Hello, {name}!"

say_hello("World")

In [13]:
import random

def uhm_what(func):
    def wrapper(*args, **kwargs):
        if random.choice([True, False]):
            return func(*args, **kwargs)
        else:
            return None
    return wrapper

In [14]:
@uhm_what
def say_hello(name):
    return f"Hello, {name}!"

for i in range(10):
    print(say_hello(i))

None
None
None
None
Hello, 4!
Hello, 5!
None
None
None
None


### Decorator Factory

In [15]:
def retry(times: int):
    def decorator_repeat(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(times):
                try:
                    return func(*args, **kwargs)
                except RuntimeError:
                    print("Failed")
                    continue
        return wrapper
    return decorator_repeat

In [16]:
@retry(times=3)
def can_fail_eg_web_request() -> str:
    if not random.choice([True, False]):
        raise RuntimeError

    return "yay"

In [17]:
result = can_fail_eg_web_request()
print(result)

yay


## ???

In [22]:
def yo_dawg_decorator(outer_function):
    @wraps(outer_function)
    def inner_decorator(inner_function):
        @wraps(inner_function)
        def wrapper(*args, **kwargs):
            print(f"Yo Dawg, I heard you like decorators,")
            print(f"so I put a decorator (@wraps) in your decorator ({outer_function.__name__})")
            print(f"so you can preserve metadata while you decorate ({inner_function.__name__}).")
            return inner_function(*args, **kwargs)
        return wrapper
    return inner_decorator

@yo_dawg_decorator
def my_decorator(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        return f(*args, **kwargs)
    return decorated_function

@my_decorator
def my_function():
    """This is my function."""
    pass

print(my_function.__name__)  # Output: my_function
print(my_function.__doc__)   # Output: This is my function.

my_function
This is my function.


In [21]:
my_function()

Yo Dawg, I heard you like decorators,
so I put a decorator (@wraps) in your decorator (my_decorator)
so you can preserve metadata while you decorate (my_function).
